In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
from decouple import config


HPC_USER = config('HPC_USER')
HPC_HOST = config('HPC_HOST')
HPC_PASSPHRASE = config('HPC_PASSPHRASE')

In [3]:
from pathlib import Path

from math_rag.infrastructure.clients import HPCClient, PBSProClient, SFTPClient


# HPC_HOST = 'login-cpu.hpc.srce.hr'  # TODO remove

sftp_client = SFTPClient(HPC_HOST, HPC_USER, HPC_PASSPHRASE)
pbs_pro_client = PBSProClient(HPC_HOST, HPC_USER, HPC_PASSPHRASE)
hpc_client = HPCClient(HPC_HOST, HPC_USER, HPC_PASSPHRASE)

In [ ]:
# TODO remove .. when not in /notebooks
source = Path('../assets/huggingface/tgi/tgi.sh')
target = Path('/lustre/home/lpanic/tgi_test.sh')

await sftp_client.upload(source, target)

In [ ]:
# TODO remove .. when not in /notebooks
source = Path('/lustre/home/lpanic/tgi_test.sh')
target = Path('../tmp/tgi.sh')

await sftp_client.download(source, target)

In [ ]:
# limited to 80 characters (79 + newline)
len('    Error_Path = x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/hello_world')

79

In [ ]:
# qstat -f 465317
queue_status = """
Job Id: 465317.x3000c0s25b0n0.hsn.hpc.srce.hr
    Job_Name = hello
    Job_Owner = lpanic@x3000c0s27b0n0.hsn.hpc.srce.hr
    resources_used.cpupercent = 0
    resources_used.cput = 00:00:00
    resources_used.mem = 0b
    resources_used.ncpus = 1
    resources_used.vmem = 0kb
    resources_used.walltime = 00:00:00
    job_state = R
    queue = cpu-single
    server = x3000c0s25b0n0.hsn.hpc.srce.hr
    Checkpoint = u
    ctime = Fri Mar 28 10:45:48 2025
    Error_Path = x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/hello_world
	/hello.e465317
    exec_host = x8000c1s4b0n1/1
    exec_vnode = (x8000c1s4b0n1:mem=1843200kb:ncpus=1:ngpus=0)
    Hold_Types = n
    Join_Path = n
    Keep_Files = oed
    Mail_Points = a
    mtime = Fri Mar 28 10:45:53 2025
    Output_Path = x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/hello_worl
	d/hello.o465317
    Priority = 0
    qtime = Fri Mar 28 10:45:48 2025
    Rerunable = True
    Resource_List.mem = 1800mb
    Resource_List.ncpus = 1
    Resource_List.ngpus = 0
    Resource_List.nodect = 1
    Resource_List.place = pack
    Resource_List.select = 1:mem=1800mb:ncpus=1:ngpus=0
    Resource_List.walltime = 48:00:00
    stime = Fri Mar 28 10:45:48 2025
    session_id = 3900810
    jobdir = /lustre/home/lpanic
    substate = 42
    Variable_List = PBS_O_HOME=/lustre/home/lpanic,PBS_O_LANG=en_US.UTF-8,
	PBS_O_LOGNAME=lpanic,
	PBS_O_PATH=/lustre/home/lpanic/.local/bin:/lustre/home/lpanic/bin:/opt
	/clmgr/sbin:/opt/clmgr/bin:/opt/sgi/sbin:/opt/sgi/bin:/usr/share/Module
	s/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/c3/bin:/op
	t/pbs/bin:/sbin:/bin,PBS_O_MAIL=/var/spool/mail/lpanic,
	PBS_O_SHELL=/bin/bash,PBS_O_HOST=x3000c0s27b0n0.hsn.hpc.srce.hr,
	PBS_O_WORKDIR=/lustre/home/lpanic/hello_world,PBS_O_SYSTEM=Linux,
	PBS_O_QUEUE=RouteQ
         = Job run at Fri Mar 28 at 10:45 on (x8000c1s4b0n1:mem=1843200kb:nc
	pus=1:ngpus=0)
    etime = Fri Mar 28 10:45:48 2025
    run_count = 1
    eligible_time = 00:00:05
    Submit_arguments = -koed hello.sh
    project = _pbs_project_default
    Submit_Host = x3000c0s27b0n0.hsn.hpc.srce.hr
"""

In [ ]:
from math_rag.infrastructure.mappings.hpcs.pbs import PBSProJobFullMapping


job = PBSProJobFullMapping.to_source(queue_status)

In [ ]:
pbs_path = Path('hello_world/hello2b.sh')
job_id = await pbs_pro_client.queue_submit(pbs_path)
job_id

In [38]:
job_id

'468995.x3000c0s25b0n0.hsn.hpc.srce.hr'

In [39]:
job = await pbs_pro_client.queue_status(job_id)
job

PBSProJob(id='468995.x3000c0s25b0n0', name='hello', user='lpanic', time=datetime.timedelta(0), state=<PBSProJobState.HELD: 'H'>, queue=<HPCQueue.GPU: 'gpu'>)

In [41]:
await pbs_pro_client.trace_job(job_id)

'Job: 468995.x3000c0s25b0n0.hsn.hpc.srce.hr\n\n03/31/2025 12:34:40  L    Formula Evaluation = 999.9785\n03/31/2025 12:34:40  L    Considering job to run\n03/31/2025 12:34:40  L    Insufficient amount of resource: Qlist \n03/31/2025 12:34:40  S    enqueuing into gpu, state Q hop 1\n03/31/2025 12:34:40  S    Job Queued at request of lpanic@x3000c0s27b0n0.hsn.hpc.srce.hr, owner = lpanic@x3000c0s27b0n0.hsn.hpc.srce.hr, job name = hello, queue = gpu\n03/31/2025 12:34:40  S    Job Modified at request of Scheduler@x3000c0s25b0n0.hsn.hpc.srce.hr\n03/31/2025 12:35:08  S    Holds u set at request of lpanic@x3000c0s27b0n0.hsn.hpc.srce.hr\n03/31/2025 12:37:43  L    Formula Evaluation = 999.9788\n03/31/2025 12:38:11  L    Formula Evaluation = 999.9788\n03/31/2025 12:38:19  L    Formula Evaluation = 999.9788\n03/31/2025 12:38:26  L    Formula Evaluation = 999.9788\n03/31/2025 12:38:55  L    Formula Evaluation = 999.9788\n03/31/2025 12:39:41  S    Holds u released at request of lpanic@x3000c0s27b0n0.

In [42]:
await pbs_pro_client.queue_delete(job_id, force=False)

In [ ]:
"""
qstat -u lpanic 468988.x3000c0s25b0n0.hsn.hpc.srce.hr

x3000c0s25b0n0.hsn.hpc.srce.hr:
                                                                 Req'd  Req'd   Elap
Job ID               Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
-------------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
468988.x3000c0s25b0* lpanic   gpu      hello      18403*   1   1  120gb 24:00 R 00:03
"""